# OpenAI Gym

## Imports

In [1]:
import numpy
import gym

In [5]:
env = gym.make('Breakout-v4')
env.reset()
episode_reward = 0
i = 0
while True:
    # print(i)
    i += 1
    action = env.action_space.sample()
    # print(env.action_space)
    # print(env.observation_space)
    # print(env.env.get_action_meanings())
    # print(action)
    _, reward, done, _ = env.step(action)
    episode_reward += reward
    if done:
        print('Reward: %s' % episode_reward)
        break

# https://www.digitalocean.com/community/tutorials/how-to-build-atari-bot-with-openai-gym
# https://colab.research.google.com/github/GiannisMitr/DQN-Atari-Breakout/blob/master/dqn_atari_breakout.ipynb
# https://www.tensorflow.org/agents/tutorials/1_dqn_tutorial

Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (210, 160, 3), uint8)
Box(0, 255, (2